# Chapter 6 - Spark SQL

Paul E. Anderson

## Ice Breaker

Good rainy day story or snow day story?

While this text can be viewed as PDF, it is most useful to have a Jupyter environment. I have an environment ready for each of you, but you can get your own local environment going in several ways. One popular way is with Anaconda (<a href="https://www.anaconda.com/">https://www.anaconda.com/</a>. Because of the limited time, you can use my server.

## Spark SQL
* Designed for structured data
* Seamlessly mix SQL queries with Spark programs
* Connects to many difference datasources: Hive, Avro, Parquet, ORC, JSON, and JDBC.
* You can join across datasources

## JSON
* JSON stands for JavaScript Object Notation
* JSON is simply a way of representing data independent of a platform
* An alternative to JSON is XML

## XML vs JSON
* Both are human-readable and machine-readable
* Most people would agree that JSON is easier to read
* JSON is faster for computers to process
* Both contain actual data and meta-information.

## Why are we talking about JSON? I thought this was Spark SQL
* Spark SQL is all about structured data
* We will therefore need to talk about different ways to represent structured data

### Example JSON file
<img src="https://static.goanywhere.com/images/tutorials/read-json/ExampleJSON2.png">

### JSON Syntax
* Collection of attribute/value pairs enclosed by curly brackets.
* The attribute is just the name of the attribute surrounded by double quotes (double and not single)
* The value can be:
    * a string (in double quotes), 
    * a number,  
    * a list of *things* of the same type (in square brackets). 
* The *thing* can be a string, a number, or a JSON expression.
* : is put between the attribute and the value and the different attribute/value pairs are separated by comma.

## JSON to Python
<img src="https://miro.medium.com/max/1484/1*uMSJMK2XLpDBfPABFZ9kTg.png" width=400>

## Back into Spark SQL

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Examine: <a href="https://corgis-edu.github.io/corgis/json/covid/">https://corgis-edu.github.io/corgis/json/covid/</a>

In [2]:
from pathlib import Path
home = str(Path.home())

### Requirement of Spark is that JSON is flat

In [3]:
import json
data = json.loads(open(f"{home}/csc-369-student/data/corgis/datasets/json/covid/covid.json").read())
open(f"{home}/csc-369-student/data/corgis/datasets/json/covid/covid_flat.json","w").write(json.dumps(data));

In [4]:
# spark is an existing SparkSession
df = spark.read.json(f"{home}/csc-369-student/data/corgis/datasets/json/covid/covid_flat.json")
# Displays the content of the DataFrame to stdout
df.show()

+--------------------+--------------+--------------------+
|                Data|          Date|            Location|
+--------------------+--------------+--------------------+
|[121, 6, 38041757...| [5, 11, 2020]|[AFG, Asia, Afgha...|
|[86, 4, 38041757,...| [4, 11, 2020]|[AFG, Asia, Afgha...|
|[95, 3, 38041757,...| [3, 11, 2020]|[AFG, Asia, Afgha...|
|[132, 5, 38041757...| [2, 11, 2020]|[AFG, Asia, Afgha...|
|[76, 0, 38041757,...| [1, 11, 2020]|[AFG, Asia, Afgha...|
|[157, 4, 38041757...|[31, 10, 2020]|[AFG, Asia, Afgha...|
|[123, 3, 38041757...|[30, 10, 2020]|[AFG, Asia, Afgha...|
|[0, 0, 38041757, ...|[29, 10, 2020]|[AFG, Asia, Afgha...|
|[113, 7, 38041757...|[28, 10, 2020]|[AFG, Asia, Afgha...|
|[199, 8, 38041757...|[27, 10, 2020]|[AFG, Asia, Afgha...|
|[65, 3, 38041757,...|[26, 10, 2020]|[AFG, Asia, Afgha...|
|[81, 4, 38041757,...|[25, 10, 2020]|[AFG, Asia, Afgha...|
|[61, 2, 38041757,...|[24, 10, 2020]|[AFG, Asia, Afgha...|
|[116, 4, 38041757...|[23, 10, 2020]|[AFG, Asia, Afgha..

### Print the schema in a tree format

In [5]:
df.printSchema()

root
 |-- Data: struct (nullable = true)
 |    |-- Cases: long (nullable = true)
 |    |-- Deaths: long (nullable = true)
 |    |-- Population: long (nullable = true)
 |    |-- Rate: double (nullable = true)
 |-- Date: struct (nullable = true)
 |    |-- Day: long (nullable = true)
 |    |-- Month: long (nullable = true)
 |    |-- Year: long (nullable = true)
 |-- Location: struct (nullable = true)
 |    |-- Code: string (nullable = true)
 |    |-- Continent: string (nullable = true)
 |    |-- Country: string (nullable = true)



### How do we grab a single column?

In [6]:
df.select("Location").show()

+--------------------+
|            Location|
+--------------------+
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
|[AFG, Asia, Afgha...|
+--------------------+
only showing top 20 rows



## Nested

In [8]:
df.select("Date.Day").show()

+---+
|Day|
+---+
|  5|
|  4|
|  3|
|  2|
|  1|
| 31|
| 30|
| 29|
| 28|
| 27|
| 26|
| 25|
| 24|
| 23|
| 22|
| 21|
| 20|
| 19|
| 18|
| 17|
+---+
only showing top 20 rows



## Filtering

In [9]:
df.filter(df['Date.Day'] > 21).show()

+--------------------+--------------+--------------------+
|                Data|          Date|            Location|
+--------------------+--------------+--------------------+
|[157, 4, 38041757...|[31, 10, 2020]|[AFG, Asia, Afgha...|
|[123, 3, 38041757...|[30, 10, 2020]|[AFG, Asia, Afgha...|
|[0, 0, 38041757, ...|[29, 10, 2020]|[AFG, Asia, Afgha...|
|[113, 7, 38041757...|[28, 10, 2020]|[AFG, Asia, Afgha...|
|[199, 8, 38041757...|[27, 10, 2020]|[AFG, Asia, Afgha...|
|[65, 3, 38041757,...|[26, 10, 2020]|[AFG, Asia, Afgha...|
|[81, 4, 38041757,...|[25, 10, 2020]|[AFG, Asia, Afgha...|
|[61, 2, 38041757,...|[24, 10, 2020]|[AFG, Asia, Afgha...|
|[116, 4, 38041757...|[23, 10, 2020]|[AFG, Asia, Afgha...|
|[135, 2, 38041757...|[22, 10, 2020]|[AFG, Asia, Afgha...|
|[15, 2, 38041757,...| [30, 9, 2020]|[AFG, Asia, Afgha...|
|[12, 3, 38041757,...| [29, 9, 2020]|[AFG, Asia, Afgha...|
|[0, 0, 38041757, ...| [28, 9, 2020]|[AFG, Asia, Afgha...|
|[35, 0, 38041757,...| [27, 9, 2020]|[AFG, Asia, Afgha..

### GroupBy

In [10]:
df.groupBy("Location.Country").count().show()

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|                Chad|  231|
|            Anguilla|  224|
|            Paraguay|  239|
|              Russia|  311|
|               Yemen|  210|
|        Burkina_Faso|  238|
|Cases_on_an_inter...|   64|
|             Senegal|  241|
|              Sweden|  311|
|         Timor_Leste|  229|
|    Marshall_Islands|    8|
|              Guyana|  236|
|             Eritrea|  229|
|              Jersey|  231|
|         Philippines|  307|
|            Djibouti|  232|
|            Malaysia|  310|
|        Sierra_Leone|  219|
|           Singapore|  311|
|        South_Africa|  243|
+--------------------+-----+
only showing top 20 rows



### But what happened to the SQL?

In [11]:
df.createOrReplaceTempView("covid") # create a temporary view so we can query our data

sqlDF = spark.sql("SELECT * FROM covid")
sqlDF.show()

+--------------------+--------------+--------------------+
|                Data|          Date|            Location|
+--------------------+--------------+--------------------+
|[121, 6, 38041757...| [5, 11, 2020]|[AFG, Asia, Afgha...|
|[86, 4, 38041757,...| [4, 11, 2020]|[AFG, Asia, Afgha...|
|[95, 3, 38041757,...| [3, 11, 2020]|[AFG, Asia, Afgha...|
|[132, 5, 38041757...| [2, 11, 2020]|[AFG, Asia, Afgha...|
|[76, 0, 38041757,...| [1, 11, 2020]|[AFG, Asia, Afgha...|
|[157, 4, 38041757...|[31, 10, 2020]|[AFG, Asia, Afgha...|
|[123, 3, 38041757...|[30, 10, 2020]|[AFG, Asia, Afgha...|
|[0, 0, 38041757, ...|[29, 10, 2020]|[AFG, Asia, Afgha...|
|[113, 7, 38041757...|[28, 10, 2020]|[AFG, Asia, Afgha...|
|[199, 8, 38041757...|[27, 10, 2020]|[AFG, Asia, Afgha...|
|[65, 3, 38041757,...|[26, 10, 2020]|[AFG, Asia, Afgha...|
|[81, 4, 38041757,...|[25, 10, 2020]|[AFG, Asia, Afgha...|
|[61, 2, 38041757,...|[24, 10, 2020]|[AFG, Asia, Afgha...|
|[116, 4, 38041757...|[23, 10, 2020]|[AFG, Asia, Afgha..

## A lot to examine

### Returns dataframe column names and data types
df.dtypes
### Displays the content of dataframe
df.show()
### Return first n rows
df.head()
### Returns first row
df.first()
### Return first n rows
df.take(5)
### Computes summary statistics
df.describe().show()
### Returns columns of dataframe
df.columns
### Counts the number of rows in dataframe
df.count()
### Counts the number of distinct rows in dataframe
df.distinct().count()

In [32]:
df.take(5)

[Row(Data=Row(Cases=121, Deaths=6, Population=38041757, Rate=3.74588377), Date=Row(Day=5, Month=11, Year=2020), Location=Row(Code='AFG', Continent='Asia', Country='Afghanistan')),
 Row(Data=Row(Cases=86, Deaths=4, Population=38041757, Rate=3.78268543), Date=Row(Day=4, Month=11, Year=2020), Location=Row(Code='AFG', Continent='Asia', Country='Afghanistan')),
 Row(Data=Row(Cases=95, Deaths=3, Population=38041757, Rate=3.78794281), Date=Row(Day=3, Month=11, Year=2020), Location=Row(Code='AFG', Continent='Asia', Country='Afghanistan')),
 Row(Data=Row(Cases=132, Deaths=5, Population=38041757, Rate=3.76691329), Date=Row(Day=2, Month=11, Year=2020), Location=Row(Code='AFG', Continent='Asia', Country='Afghanistan')),
 Row(Data=Row(Cases=76, Deaths=0, Population=38041757, Rate=3.57501889), Date=Row(Day=1, Month=11, Year=2020), Location=Row(Code='AFG', Continent='Asia', Country='Afghanistan'))]

### Convert to Pandas

In [33]:
df.toPandas()

,Data,Date,Location
0,"(121, 6, 38041757, 3.74588377)","(5, 11, 2020)","(AFG, Asia, Afghanistan)"
1,"(86, 4, 38041757, 3.78268543)","(4, 11, 2020)","(AFG, Asia, Afghanistan)"
2,"(95, 3, 38041757, 3.78794281)","(3, 11, 2020)","(AFG, Asia, Afghanistan)"
3,"(132, 5, 38041757, 3.76691329)","(2, 11, 2020)","(AFG, Asia, Afghanistan)"
4,"(76, 0, 38041757, 3.57501889)","(1, 11, 2020)","(AFG, Asia, Afghanistan)"
...,...,...,...
53585,"(0, 0, 14645473, 0.0)","(25, 3, 2020)","(ZWE, Africa, Zimbabwe)"
53586,"(0, 1, 14645473, 0.0)","(24, 3, 2020)","(ZWE, Africa, Zimbabwe)"
53587,"(0, 0, 14645473, 0.0)","(23, 3, 2020)","(ZWE, Africa, Zimbabwe)"
53588,"(1, 0, 14645473, 0.0)","(22, 3, 2020)","(ZWE, Africa, Zimbabwe)"


## Parquet
* Column oriented data format where data are stored by column rather than by row.
* Most expensive operations on hard disks are seeks
* Related data should be stored in a fashion to minimize seeks
* Many data driven tasks don't need all the columns of a row, but they do need all the data for a subset of the columns

Example of row-oriented:
<pre>
001:10,Smith,Joe,60000;
002:12,Jones,Mary,80000;
003:11,Johnson,Cathy,94000;
004:22,Jones,Bob,55000;</pre>

Example of column-oriented
<pre>
10:001,12:002,11:003,22:004;
Smith:001,Jones:002,Johnson:003,Jones:004;
Joe:001,Mary:002,Cathy:003,Bob:004;
60000:001,80000:002,94000:003,55000:004;</pre>

## And it is as easy as this to work with them in Spark

In [12]:
# DataFrames can be saved as Parquet files, maintaining the schema information.
df.write.parquet("/tmp/covid.parquet2")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("/tmp/covid.parquet2")

In [13]:
parquetFile.select('Location.Country').show()

+-----------+
|    Country|
+-----------+
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
|Afghanistan|
+-----------+
only showing top 20 rows



## Wrap-up
In addition to the Spark Core API, Spark provides convienent and flexible mechanisms to access structured data.